# Détection des émotions dans les rapports d'incidents en entreprise

## Importation des libriaires

In [ ]:
import fitz
import re
import pandas as pd
import spacy
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import pipeline

## Constitution de la abse de données

### Extraction des données textuelles dans un format .txt

In [28]:
# Importation de la librairie
import fitz 

# Charger le fichier PDF contenant les rapport 
rapport = r"D:\Projects\Missions\Rappord d'incidents - Projet 3.pdf"

# Extraction des données textuelles dans le PDF
doc = fitz.open(rapport)
text = ""
for page in doc :
    text += page.get_text('text') + "\n"

doc.close()

# Conversion en .txt

with open(r'D:\Projects\IT\Data Science & IA\Detection_Emotions_Rapports_Incidents\Text_data\raport_incident.txt', 'w', encoding='utf-8') as f :
    f.write(text)

### Ouverture du fichier .txt

In [29]:

rapport = pd.read_csv(r"D:\Projects\IT\Data Science & IA\Detection_Emotions_Rapports_Incidents\Text_data\raport_incident.txt", delimiter = "\t", encoding="utf-8")

rapport

,Rapport d'incident : Accident sur le chantier
0,Date de l'incident : 3 mars 2025
1,"Lieu : Chantier de construction, Lot 17"
2,Département concerné : Travaux et Sécurité
3,Description de l'incident :
4,Un ouvrier a glissé sur une surface mouillée n...
...,...
141,•
142,Réévaluation de la charge de travail au sein d...
143,•
144,Sensibilisation des employés à la gestion du s...


### Conversion du fichier txt en CSV

In [30]:
# Charger les textes
with open(r"D:\Projects\IT\Data Science & IA\Detection_Emotions_Rapports_Incidents\Text_data\raport_incident.txt", "r", encoding="utf-8") as f:
    texte = f.read()

# Enregistrer mon texte au format CSV
# Diviser le texte en lignes
description = texte.split("\n")
# Créer un DataFrame
incident = pd.DataFrame({'Description': description})
# Enregistrer en format CSV
incident.to_excel("incident.xlsx", index=False)

## Nouvelle base de données textuelles construite

In [31]:
textrh = pd.read_excel(r'D:\Projects\IT\Data Science & IA\Detection_Emotions_Rapports_Incidents\Text_data\incident.xlsx')
textrh

,Description
0,
1,
2,
3,
4,
...,...
202,Réévaluation de la charge de travail au sein d...
203,•
204,Sensibilisation des employés à la gestion du s...
205,l’épuisement professionnel.


In [44]:
# Charger le modèle SpaCy en français
nlp = spacy.load("fr_core_news_sm")

# Définition des mots inutils
custom_stop_word = {
    "je", "tu", "il", "elle", "nous", "vous", "ils", "elles", "mon", "ton", "son", "ma", "ta", "sa", "notre", "votre", "leur",
    "moi", "lui", "nous", "eux", "elles", "est", "sont", "était", "étaient", "être", "été", "étant", 
    "un", "une", "le", "la", "les", "ce", "cette", "ces", "celui-ci", "celui-là", "et", "mais", "ou", "si", "parce que", "donc",
    "sur", "dans", "à", "par", "avec", "au sujet de", "contre", "entre", "dans", "à travers", "par-dessus", "sous",
    "encore", "davantage", "alors", "une fois", "peut", "va", "juste", "devrait", "ferait", "pourrait", "peut-être", "pourrait", "doit","inclure", "mentionner", "déjà", "rapidement", "bientôt", "permettre", "dehors", "deuxième", "loin",
    "marché", "million", "stock", "général", "industrie", "économie", "nation", "éducation",
    "moment", "parent", "","de","pour","au","que","en",'se',"ce","chez"
}

# Nettoyage des espaces inutiles et caractères speciaux
def cleaning_text(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if( token.is_alpha or token.text.isspace()) and token.lemma_ not in custom_stop_word]
    return " ".join(tokens)

# Apllication
textrh['Inutilite_supr'] = textrh['Description'].dropna().astype(str).apply(cleaning_text)
textrh

,Description,Inutilite_supr,Entités
0,,,[]
1,,,[]
2,,,[]
3,,,[]
4,,,[]
...,...,...,...
202,Réévaluation de la charge de travail au sein d...,réévaluation charge travail sein département rh,[]
203,•,,[]
204,Sensibilisation des employés à la gestion du s...,sensibilisation employé gestion stress signe,[]
205,l’épuisement professionnel.,épuisement professionnel,[]


## Extraction des entités

In [39]:
def extraction_entity(text):
    doc =nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

textrh['Entités']=textrh['Inutilite_supr'].astype(str).apply(extraction_entity)
textrh

,Description,Inutilite_supr,Entités
0,,,[]
1,,,[]
2,,,[]
3,,,[]
4,,,[]
...,...,...,...
202,Réévaluation de la charge de travail au sein d...,réévaluation charge travail sein département rh .,[]
203,•,•,[]
204,Sensibilisation des employés à la gestion du s...,sensibilisation employé gestion stress signe,[]
205,l’épuisement professionnel.,épuisement professionnel .,[]


### Exportation des données textuelles prétraitées

In [45]:
textrh.to_excel('incident_analysis.xlsx', index=False)
print('Exportation réussie')

Exportation réussie
